# Imports

In [1]:
import selenium
import requests
import pandas as pd
import numpy as np 
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import os
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from bs4 import BeautifulSoup


## Setting chromedriver

In [5]:
import os
print(os.getcwd())

c:\Users\1176153\Downloads\github\Thesis\notebooks\0_globals


In [6]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Automatically install the correct ChromeDriver version
driver_path = ChromeDriverManager().install()

# Initialize the Service with the correct path
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

print("Using ChromeDriver at:", driver_path)  # Debugging output


Using ChromeDriver at: C:\Users\1176153\.wdm\drivers\chromedriver\win64\134.0.6998.88\chromedriver-win32/chromedriver.exe


In [12]:
# URL of the page you want to scrape
url_programs = 'https://www.novaims.unl.pt/en/education/programs/'
# Open a webpage
driver.get(url_programs)
time.sleep(5)  # Let the user actually see something



In [13]:
# Find the 'main' element on the page
main_section = driver.find_element(By.TAG_NAME, "main")

# Find all 'a' tags within the 'main' section
degree_programs = main_section.find_elements(By.TAG_NAME, "a")

# Extract the 'href' attribute from each link
degree_programs_links = [link.get_attribute('href') for link in degree_programs]



In [14]:
degree_programs_links

['https://www.novaims.unl.pt/en/',
 'https://www.novaims.unl.pt/en/education/',
 'https://www.novaims.unl.pt/en/education/programs/',
 'https://www.novaims.unl.pt/en/terms-of-use-and-privacy-policy/',
 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/#',
 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/#',
 'https://www.novaims.unl.pt/en/education/programs/doctoral-program-in-information-management/doctoral-program-in-information-management/#',
 'https://www.novaims.unl.pt/en/education/programs/workshops-and-short-duration-courses/#',
 'https://guia.unl.pt/']

In [15]:
# Filter the degree programs links
filtered_links = [link for link in degree_programs_links if link.startswith(url_programs) and link != url_programs]

# Print the filtered links
filtered_links

['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/#',
 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/#',
 'https://www.novaims.unl.pt/en/education/programs/doctoral-program-in-information-management/doctoral-program-in-information-management/#',
 'https://www.novaims.unl.pt/en/education/programs/workshops-and-short-duration-courses/#']

In [16]:
# Dictionary to store filtered links for each url_link
dict_programs_links = {}

# Loop through each filtered link
for url_link in filtered_links:
    try:
        # Check if '#' is in the URL and remove it
        if '#' in url_link:
            url_link = url_link.split('#')[0]  # Remove the '#' and everything after it

        # Navigate to the program's URL
        driver.get(url_link)
        time.sleep(2)  # Let the page load for 2 seconds

        # Find the 'main' element on the new page
        main_section = driver.find_element(By.TAG_NAME, "main")

        # Find all 'a' tags within the 'main' section of the new page
        programs = main_section.find_elements(By.TAG_NAME, "a")

        # Extract the 'href' attribute from each link
        programs_links = [link.get_attribute('href') for link in programs]

        # Create a new list to store filtered links for this specific url_link
        current_programs_links = []

        # Iterate over the programs_links and apply filtering for this specific page
        for link in programs_links:
            if isinstance(link, str) and link.startswith(url_link) and link != url_link:
                # Remove '#' if present in the link
                if '#' in link:
                    link = link.split('#')[0]
                current_programs_links.append(link)

        # Store the filtered program links in the dictionary
        dict_programs_links[url_link] = current_programs_links

        # Optionally: Store or append current_programs_links to a main collection if needed
        # filtered_programs_links.append(current_programs_links)

    except Exception as e:
        print(f"Error processing {url_link}: {e}")


In [17]:
dict_programs_links

{'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/': ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/',
  'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/'],
 'https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-programs/': ['https://www.novaims.unl.pt/en/education/programs/postgraduate-programs-and-master-degree-p

In [27]:
import pickle
# Define save path
pickle_path = r"../../data/Webscrapping/programs_links.pkl"

# Save dictionary as Pickle
with open(pickle_path, "wb") as f:
    pickle.dump(dict_programs_links, f)

print(f"Dictionary saved as Pickle: {pickle_path}")


Dictionary saved as Pickle: ../../data/Webscrapping/programs_links.pkl


## Pipeline for Bachelor Degrees

I'm ignoring at the moment every link associated to bachelor degress not related to the existing 3 bachelor degrees

In [18]:
# Check if the dictionary is not empty
if dict_programs_links:
    # Get the first key (URL) and its value (filtered links)
    first_url, first_filtered_links = next(iter(dict_programs_links.items()))
    
    # Print the first URL and filtered links
    print(f"First URL: {first_url}")
    print(f"Filtered links for the first URL: {first_filtered_links}")
    
    
    # Optionally, interact with the page after it loads
else:
    print("The dictionary is empty.")

First URL: https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/
Filtered links for the first URL: ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/']


In [38]:
# Check if the dictionary is not empty
if dict_programs_links:
    # Get the first key (URL) and its value (filtered links)
    first_url, first_filtered_links = next(iter(dict_programs_links.items()))
    
    # Print the first URL and filtered links
    print(f"First URL: {first_url}")
    print(f"Filtered links for the first URL: {first_filtered_links}")

    # Extract the first three elements from the dictionary
    first_three_programs = list(dict_programs_links.items())[:3]

    # Iterate over the first three programs
    for program_url, program_links in first_three_programs:
        # Extract the name of the bachelor degree from the URL
        bachelor_degree_name = program_url.split('/')[-2]

        # Substitute the name in the links
        links_to_extract = [
            f"https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/{bachelor_degree_name}/program/#",
            f"https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/{bachelor_degree_name}/teaching-staff/#"
        ]

        # Print the substituted links
        print(f"Substituted links for {bachelor_degree_name}: {links_to_extract}")

        # Optionally, interact with the page after it loads
else:
    print("The dictionary is empty.")


First URL: https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/
Filtered links for the first URL: ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/']
Substituted links for bachelor-s-degrees: ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/bachelor-s-degrees/program/#', 'https://www.novaims.u

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Dictionary to store filtered links for each url_link
dict_programs_links = {
    "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/": [
        "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/",
        "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/",
        "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/",
        "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/",
        "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/",
        "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/"
    ]
}

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Check if the dictionary is not empty
if dict_programs_links:
    # Get the first key (URL) and its value (filtered links)
    first_url, first_filtered_links = next(iter(dict_programs_links.items()))
    
    # Print the first URL and filtered links
    print(f"First URL: {first_url}")
    print(f"Filtered links for the first URL: {first_filtered_links}")

    # Extract the first three elements from the filtered links
    first_three_programs = first_filtered_links[:3]

    # Iterate over the first three programs
    for program_url in first_three_programs:
        # Extract the name of the bachelor degree from the URL
        bachelor_degree_name = program_url.split('/')[-2]

        # Create a file to save the extracted text from the main course page
        driver.get(program_url)
        time.sleep(5)  # Wait for the page to load

        # Extract the <main> element
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_html = main_element.get_attribute('outerHTML')

        # Parse the page with BeautifulSoup
        soup = BeautifulSoup(main_html, "html.parser")

        # Remove unwanted sections
        unwanted_sections = soup.find_all("div", class_="block")
        if len(unwanted_sections) >= 4:
            unwanted_sections[3].decompose()  # Remove only the fourth occurrence
        if len(unwanted_sections) >= 6:
            unwanted_sections[5].decompose()  # Remove the sixth occurrence
        if len(unwanted_sections) >= 7:
            unwanted_sections[6].decompose()  # Remove the seventh occurrence

        unwanted_section = soup.find("div", class_="hero-article__info")
        if unwanted_section:
            unwanted_section.decompose()

        # Remove the specified class inside the block
        block_with_aside = soup.find("div", class_="block has-aside")
        if block_with_aside:
            article_to_remove = block_with_aside.find("article", class_="content__article")
            if article_to_remove:
                article_to_remove.decompose()

        # Extract only the text, removing extra whitespace
        main_text = soup.get_text(separator="\n", strip=True)

        # Write extracted text to the file
        with open(f"{bachelor_degree_name}_main_course_extracted_text.txt", "w", encoding="utf-8") as file:
            file.write(f"Text from {program_url}:\n")
            file.write(main_text)
            file.write("\n\n")

        # Substitute the name in the links
        links_to_extract = [
            f"https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/{bachelor_degree_name}/program/#",
            f"https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/{bachelor_degree_name}/teaching-staff/#"
        ]

        # Create a file to save the extracted text from the program and teaching-staff pages
        for url in links_to_extract:
            # Open webpage
            driver.get(url)
            time.sleep(5)  # Wait for the page to load

            # Extract the <main> element
            main_element = driver.find_element(By.TAG_NAME, "main")
            main_html = main_element.get_attribute('outerHTML')

            # Parse the page with BeautifulSoup
            soup = BeautifulSoup(main_html, "html.parser")

            # Extract only the text, removing extra whitespace
            main_text = soup.get_text(separator="\n", strip=True)

            # Write extracted text to the file
            file_name = f"{bachelor_degree_name}_{url.split('/')[-2]}_extracted_text.txt"
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(f"Text from {url}:\n")
                file.write(main_text)
                file.write("\n\n")

    print("Text extraction completed and saved to files.")
else:
    print("The dictionary is empty.")

# Close the WebDriver
driver.quit()

## Main code 

In [26]:
import os
print(os.getcwd())

c:\Users\1176153\Downloads\github\Thesis\notebooks\0_globals


In [ ]:
# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)
#balchelor_dir = "../../data/Webscrapping/bachelor_degree"
# Define the correct path
balchelor_dir = r"..\..\data/Webscrapping\bachelor_degree"
# Ensure the directory exists
os.makedirs(balchelor_dir, exist_ok=True)

# Check if the dictionary is not empty
if dict_programs_links:
    # Get the first key (URL) and its value (filtered links)
    first_url, first_filtered_links = next(iter(dict_programs_links.items()))
    
    # Print the first URL and filtered links
    print(f"First URL: {first_url}")
    print(f"Filtered links for the first URL: {first_filtered_links}")

    # Extract the first three elements from the filtered links
    first_three_programs = first_filtered_links[:3]

    # Iterate over the first three programs
    for program_url in first_three_programs:
        # Extract the name of the bachelor degree from the URL
        bachelor_degree_name = program_url.split('/')[-2]

        # Create a file to save the extracted text from the main course page
        driver.get(program_url)
        time.sleep(5)  # Wait for the page to load

        # Extract the <main> element
        main_element = driver.find_element(By.TAG_NAME, "main")
        main_html = main_element.get_attribute('outerHTML')

        # Parse the page with BeautifulSoup
        soup = BeautifulSoup(main_html, "html.parser")

        # Remove unwanted sections
        unwanted_sections = soup.find_all("div", class_="block")
        if len(unwanted_sections) >= 4:
            unwanted_sections[3].decompose()  # Remove only the fourth occurrence
        if len(unwanted_sections) >= 6:
            unwanted_sections[5].decompose()  # Remove the sixth occurrence
        #if len(unwanted_sections) >= 7:
         #   unwanted_sections[6].decompose()  # Remove the seventh occurrence

        # REMOVE THE BROCHURE AND COUNTRIES SECTIONS
        unwanted_section = soup.find("div", class_="hero-article__info")
        if unwanted_section:
            unwanted_section.decompose()

        # Remove the specified articles
        articles = soup.find_all("article", class_="content__article")
        indices_to_remove = [5, 7, 8]  # 0-based index for 6th, 8th, and 9th articles
        for index in indices_to_remove:
            if index < len(articles):
                articles[index].decompose()

        # Extract only the text, removing extra whitespace
        main_text = soup.get_text(separator="\n", strip=True)


        # Create the full file path
        file_path = os.path.join(balchelor_dir, f"{bachelor_degree_name}_main_course_extracted_text.txt")

        # Write to the file
        with open(file_path, "w", encoding="utf-8") as file:
            file.write(f"Text from {program_url}:\n")
            file.write(main_text)
            file.write("\n\n")

        
        # Substitute the name in the links
        links_to_extract = [
            f"https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/{bachelor_degree_name}/program/#",
            f"https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/{bachelor_degree_name}/teaching-staff/#"
        ]

        # Create a file to save the extracted text from the program and teaching-staff pages
        for url in links_to_extract:
            # Open webpage
            driver.get(url)
            time.sleep(5)  # Wait for the page to load

            # Extract the <main> element
            main_element = driver.find_element(By.TAG_NAME, "main")
            main_html = main_element.get_attribute('outerHTML')

            # Parse the page with BeautifulSoup
            soup = BeautifulSoup(main_html, "html.parser")

            # Extract only the text, removing extra whitespace
            main_text = soup.get_text(separator="\n", strip=True)

            # Write extracted text to the file
            file_name = os.path.join(balchelor_dir, f"{bachelor_degree_name}_{url.split('/')[-2]}_extracted_text.txt")
            with open(file_name, "w", encoding="utf-8") as file:
                file.write(f"Text from {url}:\n")
                file.write(main_text)
                file.write("\n\n")

    print("Text extraction completed and saved to files.")
else:
    print("The dictionary is empty.")

# Close the WebDriver
driver.quit()

First URL: https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/
Filtered links for the first URL: ['https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-management/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/information-systems/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-of-access-and-entry-change-of-institution-course-pair-academic-year-2024-2025/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/applications-to-the-special-contest-for-over-23-years-old-access-entry-academic-year-2024-2025/', 'https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/international-students/']
Text extraction completed and saved to files.


In [61]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# URL for the Data Science program
data_science_url = "https://www.novaims.unl.pt/en/education/programs/bachelor-s-degrees/data-science/"

# Open webpage
driver.get(data_science_url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")
main_html = main_element.get_attribute('outerHTML')

# Parse the page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")


# Remove unwanted sections
unwanted_sections = soup.find_all("div", class_="block")
if len(unwanted_sections) >= 4:
    unwanted_sections[3].decompose()  # Remove only the fourth occurrence'
    

if len(unwanted_sections) >= 6:
    unwanted_sections[5].decompose()  # Remove the sixth occurrence'
    
if len(unwanted_sections) >= 7:
    unwanted_sections[6].decompose()  # Remove the seventh occurrence


unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()

# Remove the specified articles
articles = soup.find_all("article", class_="content__article")
indices_to_remove = [5, 7, 8]  # 0-based index for 6th, 8th, and 9th articles
for index in indices_to_remove:
    if index < len(articles):
        articles[index].decompose()

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print("Extracted Text from Data Science Main Course Page:")
print(main_text)

# Close the WebDriver
driver.quit()

Extracted Text from Data Science Main Course Page:
Data Science
Degree in
Data Science
en
Education
Programs
Bachelor's Degrees
Data Science
In the Bachelor´s Degree in Data Science, students learn the most modern techniques of artificial intelligence and machine learning to analyze large volumes of data (Big Data).
They will become true data scientists - considered the sexiest profession of the 21
st
century by the Harvard Business Review.
The main objective of this course is to train future professionals capable of understanding, developing and using models, algorithms and the most advanced techniques in data science, to analyze and extract knowledge from Big Data.
The 3
rd
phase of applications under the International Student Statute for the 2025/26 academic year are open from February 26
th
to March 27
th
, 2025.
Duration
3 years (6 semesters)
Timetable
Daytime
Start
September 2025
Career Opportunities
The Bachelor´s Degree in Data Science allows a quick integration in the most var

## FIN


In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)



Ciência de Dados
Licenciatura em
Ciência de Dados
pt
Ensino
Cursos
Licenciaturas
Ciência de Dados
A
Licenciatura em Ciência de Dados
proporciona uma experiência de aprendizagem sem igual, onde os alunos se familiarizam com as mais recentes técnicas de inteligência artificial e machine learning. Esta formação foi especificamente desenhada para habilitar os alunos a analisar grandes volumes de dados, conhecidos como Big Data e a extrair conhecimento relevante.
Neste curso, preparamos os alunos para se tornarem autênticos data scientists, uma profissão que a Harvard Business Review designou como a 'mais atrativa do século XXI'. Orientamos para um futuro dinâmico e em constante mudança, fornecendo as ferramentas para compreender, desenvolver e utilizar os modelos e algoritmos mais avançados na área da ciência de dados.
O principal objetivo da NOVA IMS é formar profissionais capazes de ir além do convencional, usando técnicas pioneiras de ciência de dados para transformar Big Data em insigh

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Close the WebDriver
driver.quit()

# Parse with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Remove the unwanted section
unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()  # Removes the element from the soup
    

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text
print(main_text)

# Optionally, save to a text file
with open("ciencia_de_dados_filtered_text.txt", "w", encoding="utf-8") as file:
    file.write(main_text)



Ciência de Dados
Licenciatura em
Ciência de Dados
pt
Ensino
Cursos
Licenciaturas
Ciência de Dados
A
Licenciatura em Ciência de Dados
proporciona uma experiência de aprendizagem sem igual, onde os alunos se familiarizam com as mais recentes técnicas de inteligência artificial e machine learning. Esta formação foi especificamente desenhada para habilitar os alunos a analisar grandes volumes de dados, conhecidos como Big Data e a extrair conhecimento relevante.
Neste curso, preparamos os alunos para se tornarem autênticos data scientists, uma profissão que a Harvard Business Review designou como a 'mais atrativa do século XXI'. Orientamos para um futuro dinâmico e em constante mudança, fornecendo as ferramentas para compreender, desenvolver e utilizar os modelos e algoritmos mais avançados na área da ciência de dados.
O principal objetivo da NOVA IMS é formar profissionais capazes de ir além do convencional, usando técnicas pioneiras de ciência de dados para transformar Big Data em insigh

In [7]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode (no UI)
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service, options=chrome_options)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Scroll to ensure content is fully loaded
driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
time.sleep(3)  # Allow content to load

# Extract elements with the desired class
elements = driver.find_elements(By.CLASS_NAME, "slider-quote")

# Get text content from each element
extracted_texts = [element.text.strip() for element in elements if element.text.strip()]

# Close the WebDriver
driver.quit()

# Print extracted text
for text in extracted_texts:
    print(text)




Testemunhos de Alunos - Licenciaturas
ILONA NACU
Estudante de 3º ano de Licenciatura
2 / 5
A Licenciatura em Ciência de Dados é perfeita para quem procura uma mistura de programação, matemática e estatística, tudo em quantidades ideais. O curso é prático e bem direcionado, mostrando-nos como será trabalhar no mercado real, introduzindo-nos tópicos atuais e aprofundando vários ramos com a progressão da licenciatura. Os tópicos dos projetos são diversos, mostrando-nos o quão versátil a nossa área realmente pode ser. Os professores estão a par das tendências e novidades do setor e são acessíveis para dúvidas e sugestões. Apesar de estar satisfeira com a licenciatura desde o início do meu percurso académico, ao longo dos semestres, o curso conseguiu captar-me mais e mais.
O que faz um Cientista de Dados?
MAURO CASTELLI | COORDENADOR DA LICENCIATURA
Sorry, your browser doesn't support embedded videos.
Ver Vídeo


In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

In [16]:


# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Find the button link dynamically
button = soup.find("a", class_="content__link arrow-animate")
if button:
    new_url = button['href']
    if not new_url.startswith("http"):
        new_url = "https://www.novaims.unl.pt" + new_url

    # Navigate to the new URL
    driver.get(new_url)
    time.sleep(5)  # Wait for the new page to load

    # Extract the new page's <main> element
    new_main_element = driver.find_element(By.TAG_NAME, "main")
    new_main_html = new_main_element.get_attribute('outerHTML')

    # Parse the new page with BeautifulSoup
    new_soup = BeautifulSoup(new_main_html, "html.parser")

    # Extract only the text from the new page, removing extra whitespace
    new_main_text = new_soup.get_text(separator="\n", strip=True)

    # Print extracted text from the new page
    print("\nNew Page Text:")
    print(new_main_text)

# Close the WebDriver
driver.quit()

# Remove unwanted sections
unwanted_sections = soup.find_all("div", class_="block")
if len(unwanted_sections) >= 4:
    unwanted_sections[3].decompose()  # Remove only the fourth occurrence
if len(unwanted_sections) >= 6:
    unwanted_sections[5].decompose()  # Remove the sixth occurrence
if len(unwanted_sections) >= 7:
    unwanted_sections[6].decompose()  # Remove the seventh occurrence

unwanted_section = soup.find("div", class_="hero-article__info")
if unwanted_section:
    unwanted_section.decompose()

# Extract only the text, removing extra whitespace
main_text = soup.get_text(separator="\n", strip=True)

# Print extracted text from the original page
print("Original Page Text:")
print(main_text)

# Optionally, save to a text file
with open("ciencia_de_dados_filtered_text_3.txt", "w", encoding="utf-8") as file:
    file.write(main_text)



New Page Text:
Plano de Estudos - Ciência de Dados
Plano de Estudos
pt
Ensino
Cursos
Licenciaturas
Ciência de Dados
Plano de Estudos - Ciência de Dados
1º ano - Semestre Outono
resumo do conteudo da tabela
Unidades Curriculares
Duração
Tipo
Carga Horária de Contato
Carga Horária Total
ECTS
Álgebra Linear
Semestral
Obrigatória
T - 22,5 | PL - 22,5 | OT - 7,5
112
4
Arquiteturas de Computadores
Semestral
Obrigatória
TP - 45 | OT - 5
112
4
Aspetos Fundamentais da Ciência de Dados
Semestral
Obrigatória
TP - 45 | OT - 5
112
4
Introdução à Programação
Semestral
Obrigatória
TP - 67,5 | OT - 7,5
196
7
Sistemas de Informação
Semestral
Obrigatória
TP - 64,5 | S - 3 | OT - 7,5
168
6
Mostrar legendas
T – Ensino Teórico;
TP – Ensino Teórico-Prático;
TC – Trabalho de Campo;
PL – Ensino Prático e Laboratorial;
S – Seminário;
OT – Orientação Tutorial;
** – UC em funcionamento nos dois semestres;
1º ano - Semestre Primavera
resumo do conteudo da tabela
Unidades Curriculares
Duração
Tipo
Carga Horária d

### getting every link inside div class="block has-aside":

In [17]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Find all links inside the div with class "block has-aside"
block_div = soup.find("div", class_="block has-aside")
links = block_div.find_all("a", href=True)

# Print all found links
for link in links:
    print(link['href'])

# Close the WebDriver
driver.quit()

https://candidaturas.novaims.unl.pt/
https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/estudante-internacional/acesso-estudante-internacional/
https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/plano-de-estudos-ciencia-de-dados/#
mailto:americo.rio@novaims.unl.pt
https://www.novaims.unl.pt/pt/a-nova-ims/docentes/d/151
mailto:cristina@novaims.unl.pt
https://www.novaims.unl.pt/pt/a-nova-ims/docentes/d/128
mailto:avaranda@novaims.unl.pt
https://www.novaims.unl.pt/pt/a-nova-ims/docentes/d/542
mailto:ajrsantos@novaims.unl.pt
https://www.novaims.unl.pt/pt/a-nova-ims/docentes/d/625
mailto:bdamasio@novaims.unl.pt
https://www.novaims.unl.pt/pt/a-nova-ims/docentes/d/61
mailto:calbuquerque@novaims.unl.pt
https://www.novaims.unl.pt/pt/a-nova-ims/docentes/d/180
https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/corpo-docente/#
https://www.novaims.unl.pt/pt/ensino/alunos/bolsas/
/media/3z5gtlge/academic_calendar_bachelor_20242025.pdf
http://cur

In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
from webdriver_manager.chrome import ChromeDriverManager

# Setup WebDriver
driver_path = ChromeDriverManager().install()
service = Service(driver_path)
driver = webdriver.Chrome(service=service)

# Target URL
url = "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/"

# Open webpage
driver.get(url)
time.sleep(5)  # Wait for the page to load

# Extract the <main> element
main_element = driver.find_element(By.TAG_NAME, "main")

# Get the inner HTML of <main>
main_html = main_element.get_attribute('outerHTML')

# Parse the original page with BeautifulSoup
soup = BeautifulSoup(main_html, "html.parser")

# Find specific links
links_to_extract = [
    "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/plano-de-estudos-ciencia-de-dados/#",
    "https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/corpo-docente/#"
]

# Extract text from each specified link and save to different files
for link in links_to_extract:
    # Navigate to the new URL
    driver.get(link)
    time.sleep(5)  # Wait for the new page to load

    # Extract the new page's <main> element
    new_main_element = driver.find_element(By.TAG_NAME, "main")
    new_main_html = new_main_element.get_attribute('outerHTML')

    # Parse the new page with BeautifulSoup
    new_soup = BeautifulSoup(new_main_html, "html.parser")

    # Extract only the text from the new page, removing extra whitespace
    new_main_text = new_soup.get_text(separator="\n", strip=True)

    # Determine the filename based on the link
    if "plano-de-estudos-ciencia-de-dados" in link:
        filename = "plano_de_estudos_ciencia_de_dados.txt"
    elif "corpo-docente" in link:
        filename = "corpo_docente_ciencia_de_dados.txt"
    else:
        continue

    # Save the text content to a file
    with open(filename, "w", encoding="utf-8") as file:
        file.write(new_main_text)

    # Print extracted text from the new page
    print(f"\nText from {link}:")
    print(new_main_text)

# Close the WebDriver
driver.quit()


Text from https://www.novaims.unl.pt/pt/ensino/cursos/licenciaturas/ciencia-de-dados/plano-de-estudos-ciencia-de-dados/#:
Plano de Estudos - Ciência de Dados
Plano de Estudos
pt
Ensino
Cursos
Licenciaturas
Ciência de Dados
Plano de Estudos - Ciência de Dados
1º ano - Semestre Outono
resumo do conteudo da tabela
Unidades Curriculares
Duração
Tipo
Carga Horária de Contato
Carga Horária Total
ECTS
Álgebra Linear
Semestral
Obrigatória
T - 22,5 | PL - 22,5 | OT - 7,5
112
4
Arquiteturas de Computadores
Semestral
Obrigatória
TP - 45 | OT - 5
112
4
Aspetos Fundamentais da Ciência de Dados
Semestral
Obrigatória
TP - 45 | OT - 5
112
4
Introdução à Programação
Semestral
Obrigatória
TP - 67,5 | OT - 7,5
196
7
Sistemas de Informação
Semestral
Obrigatória
TP - 64,5 | S - 3 | OT - 7,5
168
6
Mostrar legendas
T – Ensino Teórico;
TP – Ensino Teórico-Prático;
TC – Trabalho de Campo;
PL – Ensino Prático e Laboratorial;
S – Seminário;
OT – Orientação Tutorial;
** – UC em funcionamento nos dois semestres;
